### 문제 1-2: 2단계 체인 만들기 - 영화 추천 시스템 해답

## 환경 설정

In [1]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [2]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

In [3]:
# 필요한 라이브러리 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

## 문제 2 해답 구현

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인

In [4]:
# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [5]:
# 2단계: 영화 줄거리 요약 프롬프트
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [6]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie_info로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

 추천된 영화: Drama 장르의 한국영화로 '올드보이' (2003)를 추천합니다. '올드보이'는 한국 영화계의 거장 박찬욱 감독의 작품으로, 복잡한 스토리와 뛰어난 연기로 많은 사랑을 받았습니다.

이 영화를 추천하는 이유는, 주인공의 비극적인 과거와 그로 인한 복수심이 스토리의 중심을 이루고 있어 드라마 장르에 적합합니다. 또한, 영화의 배경과 인물들의 심리적 변화를 섬세하게 표현하여, 관객에게 깊은 감동을 선사합니다. 

'올드보이'는 한국 영화의 대표작 중 하나로, 드라마 장르를 좋아하는 분들에게 강력히 추천합니다.


In [7]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

영화 추천 시스템

입력 장르: Drama

1단계 - 영화 추천 결과:
**영화:** '올드보이 (2003)'

**추천 이유:** '올드보이'는 한국 드라마/스릴러 영화로, 박찬욱 감독의 대표작 중 하나입니다. 이 영화는 복잡한 줄거리와 강렬한 연출로 유명하며, 한국 영화의 정수를 경험할 수 있는 작품입니다. 주연인 최민식, 유해진, 강혜정 등이 뛰어난 연기를 펼쳤으며, 이 영화는 다양한 국제 영화제에서 수상하며 세계적으로도 인정받았습니다. 특히, 이 영화는 한국 드라마의 특징인 감정적인 깊이와 긴장감을 잘 표현하여, 드라마 팬들에게 강력히 추천합니다.

---

2단계 - 영화 줄거리 요약:
다음은 영화 '올드보이'의 등장인물과 줄거리 요약입니다.

*   **등장인물**: 오대수(최민식), 이우진(유해진), 미도(김혜선), 장면(신하균) 등
*   **줄거리**: 오대수는 어느 날 갑자기 납치되어 15년간 비밀 감금된 채 지내게 됩니다. 오대수는 자신의 납치와 감금의 진상을 밝히고 복수를 다짐합니다. 오대수는 감금된 동안 미도와 장면의 존재를 알게 되며, 이들과의 인연을 통해 자신의 과거를 조금씩 기억해 가기 시작합니다. 오대수는 자신의 과거와 납치의 진실을 추적하며, 이우진과 대결하게 됩니다.


## 추가 테스트

In [8]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

다른 장르로 테스트 - 로맨스:

영화 추천:
*   **영화 제목:** <사랑의 달콤함> 
*   **왜 이 영화를 추천하는지 이유:** 이 영화는 로맨스 장르의 전형적인 클리셰를 탈피하여 새로운 이야기를 전달합니다. 주인공들 사이의 감정 변화를 섬세하게 표현하며, 현실적인 사랑과 이별의 과정을 보여줍니다. 또한, 배우들의 연기가 돋보이며, 특히 두 주연 배우의 케미스트리가 뛰어나 많은 사랑을 받았습니다. 이 영화는 사랑에 대한 깊은 통찰력을 제공하며, 관객들에게 감동과 공감을 불러일으킵니다.

줄거리 요약:
Here's a 10-line summary of the movie "사랑해, 사랑해, 사랑해" (2007):

*   가수 지망생 오수(정유미)는 대학교 축제에서 우연히 가수 동방신기 멤버 최시원(김수현)을 만나게 됩니다.
*   최시원은 오수를 '내 운명의 연인'으로 생각하고, 오수를 쫓아 다니기 시작합니다.
*   하지만, 오수는 시원의 팬이 되면서도 정작 자신과 시원이 인연인지 아닌지 확신이 서지 않습니다.
*   최시원은 오수를 만나기 위해 가수 데뷔를 포기하고, 오수를 쫓아 다닙니다.
*   오수와 시원의 관계가 밝혀져 오수의 마음과 시원의 마음을 얻기 위해 노력하는 영화입니다.
*   이 영화는 사랑과 우정, 음악이 조화롭게 어우러진 로맨스 영화입니다.
*   김수현과 정유미의 케미스트리가 돋보이며, 동방신기의 멤버가 등장하는 점도 흥미롭습니다.
*   영화는 감동적인 장면과 웃음 가득한 장면이 번갈아 가며, 관객에게 즐거운 경험을 제공합니다.
*   사랑과 음악에 대한 열정으로 가득 찬 두 주인공이 서로를 향한 사랑을 이루기 위해 노력하는 과정이 아름답고 감동적입니다.
*   영화 "사랑해, 사랑해, 사랑해"는 2007년에 개봉된 한국 로맨스 영화입니다.


## 체인 연결 패턴 설명

In [9]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

체인 연결 방식 설명:

1단계 체인:
movie_recommendation_prompt | llm | StrOutputParser()

2단계 체인:
{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()

핵심 포인트:
- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달
- movie 키를 통해 데이터 흐름 제어
- LCEL을 통해 직관적인 파이프라인 구성


## 학습 정리

### 문제 2에서 배운 핵심 개념
1. **ChatPromptTemplate**: 시스템과 사용자 메시지 구조화
2. **다단계 체인**: 여러 단계로 나누어 복잡한 작업 처리
3. **체인 간 데이터 전달**: {"key": previous_chain} 패턴
4. **단계별 결과 확인**: 각 체인의 중간 결과 출력

### 구현 완료 사항
- ✅ 2단계 체인 구현
- ✅ ChatPromptTemplate 사용
- ✅ 체인 간 데이터 전달
- ✅ 영화 추천 기능
- ✅ 줄거리 요약 기능
- ✅ 각 단계별 결과 출력
- ✅ 다양한 장르 테스트

### 체인 연결의 핵심 패턴
```python
# 기본 패턴
chain1 = prompt1 | llm | parser
chain2 = {"input_key": chain1} | prompt2 | llm | parser
```

이 패턴을 통해 복잡한 작업을 단계별로 분해하여 처리할 수 있습니다.